<a href="https://colab.research.google.com/github/shank10/Gemma/blob/main/GemmaTrainingToBetterCoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install wandb -qU
!pip install peft
!pip install accelerate
!pip install evaluate
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
!pip install transformers
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.4 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments,EvalPrediction,pipeline
from peft import LoraConfig, TaskType, get_peft_model , PeftModel, LoraModel
from accelerate.utils import release_memory
from accelerate import Accelerator
from datasets import Dataset
import evaluate
import accelerate

from sklearn.metrics import f1_score
from typing import Dict

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from trl import SFTTrainer

import torch
import torch.nn

import os
os.environ['TOKENIZERS_PARALLELISM'] = "FALSE"
import warnings
warnings.filterwarnings("ignore")
import wandb
import gc
gc.collect()
wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
code = pd.read_csv("/content/drive/MyDrive/Gemma/Instruction/train.csv")
code = code.sample(frac = 0.1,random_state = 101,ignore_index=True)
qa = pd.read_csv("/content/drive/MyDrive/Gemma/Glaive/train.csv",names = ['output','instruction'])
qa = qa.sample(frac = 0.01,random_state = 101,ignore_index=True)

code.info()
qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1861 entries, 0 to 1860
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  1861 non-null   object
 1   input        1309 non-null   object
 2   output       1861 non-null   object
 3   prompt       1861 non-null   object
dtypes: object(4)
memory usage: 58.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361 entries, 0 to 1360
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   output       1361 non-null   object
 1   instruction  1361 non-null   object
dtypes: object(2)
memory usage: 21.4+ KB


In [6]:
def preprocess_data(example):
    template = f"""Instruction:\n{example['instruction']}\n\nResponse:\n{example['output']}"""
    return template

code['template'] = code.apply(preprocess_data,axis=1)
qa['template'] = qa.apply(preprocess_data,axis=1)

dataset = Dataset.from_pandas(pd.concat([code[['template']],qa[['template']]])).train_test_split(test_size = 0.2)

# Example Printing
print(code['template'].iloc[1])

Instruction:
Write a Python script to print all prime numbers between two numbers, a and b (where a<b).

Response:
for num in range(a, b+1): 
   if num > 1: 
       for i in range(2, num): 
           if (num % i) == 0: 
               break
       else: 
           print(num)


In [12]:
model_structure = """This is the structure of Gemma Model,


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=256000, bias=False)
)"""
print(model_structure)

This is the structure of Gemma Model,


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_at

In [7]:
!pip install immutabledict

In [8]:
# Choose variant and machine type
VARIANT = '2b-it'
MACHINE_TYPE = 'cuda'

In [9]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [10]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'


Attaching model 'google/gemma/pyTorch/2b-it' to your Colab notebook...


In [11]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 107 (delta 29), reused 17 (delta 17), pack-reused 68
Receiving objects: 100% (107/107), 2.14 MiB | 11.12 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [12]:
import sys

sys.path.append('gemma_pytorch')


In [13]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [14]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()


In [17]:
# Generate with one request in chat mode

# Chat templates
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

# Sample formatted prompt
prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='What is a good place for travel in the US?'
    )
    + MODEL_CHAT_TEMPLATE.format(prompt='California.')
    + USER_CHAT_TEMPLATE.format(prompt='What can I do in California?')
    + '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=200,
)
# Generate sample
model.generate(
    'Write program to find factorial of number.',
    device=device,
    output_len=200,
)

Chat prompt:
 <start_of_turn>user
What is a good place for travel in the US?<end_of_turn>
<start_of_turn>model
California.<end_of_turn>
<start_of_turn>user
What can I do in California?<end_of_turn>
<start_of_turn>model



'\n\n```python\n# This Python program finds the factorial of a number.\n\ndef factorial(n):\n  """\n  This function calculates the factorial of a non-negative integer n.\n\n  Args:\n    n: The integer whose factorial is to be calculated.\n\n  Returns:\n    The factorial of n.\n  """\n\n  # Initialize the factorial to 1.\n  factorial = 1\n\n  # Iterate through all the numbers from 1 to n.\n  for i in range(1, n + 1):\n    # Multiply the factorial of i by the factorial of n.\n    factorial *= i\n\n  # Return the factorial of n.\n  return factorial\n\n\n# Get the input from the user.\nn = int(input("Enter a non-negative integer: "))\n\n# Calculate the factorial of n.\nfactorial = factorial(n)\n\n# Print the factorial of n.\nprint(f"The factorial'

In [19]:
#base_model = model
base_model = "IngeniousArtist/Gemma-1b-v3"
lora_config = LoraConfig(
        lora_dropout=0.1,
        r=8,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='fp4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config)

loraModel = LoraModel(model= model,config =lora_config,adapter_name = "adapter")

OSError: IngeniousArtist/Gemma-1b-v3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(loraModel)

In [ ]:
torch.cuda.empty_cache()
trainer.train()


In [ ]:
lora_adapter_path = "LoraAdapter"
trainer.model.save_pretrained(lora_adapter_path)


In [ ]:
# Release Memory
trainer, model, = release_memory(trainer, model)
gc.collect()
torch.cuda.empty_cache()


In [ ]:
final_model = "Final_Model"
# Loading Base Model
model = AutoModelForCausalLM.from_pretrained(base_model,device_map='auto', torch_dtype=torch.float16)

# Lodaing  Base Model with LoRA adapters.
peft_model = PeftModel.from_pretrained(model,lora_adapter_path,device_map='auto', torch_dtype=torch.float16)

# Merging and Saving Model
model = peft_model.merge_and_unload(progressbar = True)
model.save_pretrained(final_model)
tokenizer.save_pretrained(final_model)


In [ ]:
model = release_memory(model)

In [ ]:
peft_pipe = pipeline("text-generation",final_model, model_kwargs={"torch_dtype": torch.float16},
    device_map='auto',
    max_new_tokens=512)


In [ ]:
def get_output(question):
    prompt = f"Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.\n\nInstruction:\n{question}\n\nResponse:\n"
    out = peft_pipe(prompt,
     do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True)

    return out


In [ ]:
out = get_output("Write program to find factorial of number")
print(out[0]['generated_text'])


In [ ]:
out = get_output("Explain how can an AI classify a given sentence as a declaration, an expression, or a statement?")
print(out[0]['generated_text'])


In [ ]:
out = get_output("I need to write a bash script that compares two arguments and prints a message if the first argument is greater than the second argument. Can someone help me with the code")
print(out[0]['generated_text'])


In [ ]:
out = get_output("How can I optimize a TensorFlow neural network to improve its predictive accuracy?")
print(out[0]['generated_text'])
